In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

## Function For Dept Names and Links

In [2]:
def get_page_source(url):
    driver = webdriver.Chrome()
    driver.get(url)
    WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'department-title')))

    page_source = driver.page_source
    return page_source

def get_DeptName_DeptLink(html_data):
    soup = BeautifulSoup(html_data)
    div_tags_list = soup.find_all('div', class_ = 'card-content')
    dept_names, dept_links = [], []
    for div_tag in div_tags_list:
        dept_names.append(div_tag.h3.text)
        dept_links.append(div_tag.a['href'])
    
    data = {
        'Dept Name': dept_names,
        'Dept Link': dept_links
    }
    return data

def save_dept_name_and_links(url):
    page_html_data = get_page_source(url)
    dept_data = get_DeptName_DeptLink(page_html_data)
    
    df = pd.DataFrame(dept_data)
    df.to_csv('Department_data.csv', index = None)
    print('Data Saved')

In [ ]:
save_dept_name_and_links('https://www.nitw.ac.in/departments')

# Teaching Faculty data


## Function For Getting Data of Each Professor in 'A Dept'

In [ ]:
# required functions

def get_page_source(url):
    driver = webdriver.Chrome()
    driver.get(url)
    WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'data')))

    page_source = driver.page_source
    return page_source

def parse_email(email):
    email = email.replace('[AT]', '@')
    email = email.replace('[DOT]', '.')
    return email

def get_contact_info(div_tag):
    contact_tag = div_tag.find('p', class_ = 'contact-info')
    if contact_tag is None:
        return ('', '')
    contact_tag = contact_tag.text.strip().split()
    if len(contact_tag) < 1:
        return ('', '')
    if len(contact_tag) == 1:
        return ('', contact_tag[0])
    else:
        return (contact_tag[0], contact_tag[-1])

def initialize_info_parameters(link, pos, contact, email, res_area):
    para = { 
        'Profile Link': link, 
        'Position':pos, 
        'Contact No.': contact, 
        'Email ID':email,
        'Research Area':res_area
    }
    return para

In [ ]:
# function to get all the data
def prof_info(page_source):
    soup = BeautifulSoup(page_source)
    all_divs_list = soup.find_all('div', class_= 'data')
    
    all_prof_data = []
    for data_div in all_divs_list:

        prof_name = data_div.a.text if data_div.a is not None else ''
        prof_profile_link = data_div.a['href'] if data_div.a is not None else ''
        position = data_div.span.text if data_div.span is not None else ''
        contact_info = get_contact_info(data_div)
        contact_no = contact_info[0]
        email_id = contact_info[1]
        email_id = parse_email(email_id)
        contact_tag = data_div.find('p', class_ = 'contact-info')
        area_of_research = contact_tag.next_sibling.text if contact_tag is not None else ''
        
        details = initialize_info_parameters(prof_profile_link, position, contact_no, email_id, area_of_research)
        all_prof_data.append((prof_name, details))
    return dict(all_prof_data)

In [ ]:
page_source = get_page_source('https://www.nitw.ac.in/cse')
data = prof_info(page_source)

## Faculty Data of 'All Dept' 

In [ ]:
# required functions
def get_page_source_for_DEPT(url):
    driver = webdriver.Chrome()
    driver.get(url)
    WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'department-title')))

    page_source = driver.page_source
    return page_source

def get_dept_link(html_data):
    dept_links = []
    dept_link_tag_list = soup.find_all('a', class_ = 'deptviewmore')
    for tag in dept_link_tag_list:
        dept_links.append(tag['href'])
    return dept_links

def get_page_source_for_DATA(url):
    driver = webdriver.Chrome()
    driver.get(url)
    WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'data')))

    page_source = driver.page_source
    return page_source

def parse_email(email):
    email = email.replace('[AT]', '@')
    email = email.replace('[DOT]', '.')
    return email

def get_contact_info(div_tag):
    contact_tag = div_tag.find('p', class_ = 'contact-info')
    if contact_tag is None:
        return ('', '')
    contact_tag = contact_tag.text.strip().split()
    if len(contact_tag) < 1:
        return ('', '')
    if len(contact_tag) == 1:
        return ('', contact_tag[0])
    else:
        return (contact_tag[0], contact_tag[-1])

def initialize_info_parameters(link, pos, contact, email, res_area):
    para = { 
        'Profile Link': link, 
        'Position':pos, 
        'Contact No.': contact, 
        'Email ID':email,
        'Research Area':res_area
    }
    return para

def prof_info(page_source):
    soup = BeautifulSoup(page_source)
    all_divs_list = soup.find_all('div', class_= 'data')
    
    all_prof_data = []
    for data_div in all_divs_list:

        prof_name = data_div.a.text if data_div.a is not None else ''
        prof_profile_link = data_div.a['href'] if data_div.a is not None else ''
        position = data_div.span.text if data_div.span is not None else ''
        contact_info = get_contact_info(data_div)
        contact_no = contact_info[0]
        email_id = contact_info[1]
        email_id = parse_email(email_id)
        contact_tag = data_div.find('p', class_ = 'contact-info')
        area_of_research = contact_tag.next_sibling.text if (contact_tag is not None and contact_tag.next_sibling is not None) else ''
        
        details = initialize_info_parameters(prof_profile_link, position, contact_no, email_id, area_of_research)
        all_prof_data.append((prof_name, details))
    return dict(all_prof_data)


def parse_email(email):
    email = email.replace('[AT]', '@')
    email = email.replace('[DOT]', '.')
    return email


def parse_email2(email):
    email = email.replace('[@]', '@')
    email = email.replace('[dot]', '.')
    return email

def get_correct_email(email):
    if '[@]' in email or '[dot]' in email:
        return parse_email2(email)
    else:
        return email
    
def save_whole_data_of_prof_of_A_DEPT(url):
    page_source = get_page_source_for_DATA(url)
    data = prof_info(page_source)
    
    df = pd.DataFrame(data.values(), index = data.keys())
    
    # removing 'prof.' from front of name
    df['Professor Name']= df.index
    df['Professor Name'] = df['Professor Name'].apply(lambda x: x.strip().split('Prof.')[-1].strip())
    df.index = df['Professor Name']
    
    # removing newly made col 
    df.drop('Professor Name', axis = 1, inplace = True)
    
    # correcting format of email
    df['Email ID'] = df['Email ID'].apply(get_correct_email)
    
    # remove the last treasurer and faculty advisor from list
    df = df[df["Position"].str.strip() != "Treasurer"]
    df = df[df['Position'].str.strip() != 'Faculty Advisor']
    
    # Saving the data of each prof
    dept_name = url.strip().split('/')[-1]
    
    if not os.path.exists('Faculty_Data'):
        os.mkdir('Faculty_Data')
    file_path = os.path.join('Faculty_Data', dept_name)
    if os.path.exists(file_path+'.csv'):
        print(f'"{dept_name}" department data file already exists. Skipping......')
        return 
    else:
        df.to_csv(file_path+'.csv')
        print(f'Faculty data has be Successfully written into "{dept_name}.csv" file')
        return 

In [ ]:
'''
    MAIN FUNCTION
'''
def Prof_data_for_each_Dept(url):
    # 1. Get the links of all dept
    page_source = get_page_source_for_DEPT(url)
    dept_links = get_dept_link(page_source)
    
    for department_link in dept_links:
        save_whole_data_of_prof_of_A_DEPT(department_link)
        
    print('-------------------------------------------------------------')
    print('All Data Written Successfully')

In [ ]:
main_link = 'https://www.nitw.ac.in/departments'
Prof_data_for_each_Dept(main_link)

# Reasearch Faculty Data

## Save Data of All Profs of 'A Dept' in csv files

In [ ]:
def get_page_source_for_DATA2(url):
    driver = webdriver.Chrome()
    driver.get(url)
    WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'col-md-6')))

    page_source = driver.page_source
    return page_source

def get_contact_info(li):
    if len(li) == 0:
        return ['', '']
    if len(li) == 1:
        if li[0].isdigit():
            return [li[0], '']
        else:
            return ['', li[1]]
    else:
        return [li[0], li[1]]

def get_area_and_profData(url):
    page_source = get_page_source_for_DATA2(url)
    soup = BeautifulSoup(page_source)
    
    div_tag = soup.find_all('div', class_ = 'row academic_group')
    div_tag = div_tag[1:]
    
    final_data = []
    for tag in div_tag:
        area = tag.find_all('div', class_ = 'col-md-12')[0].header.h4.text
        
        prof_tag_list = tag.find_all('div', class_ = 'ml-3 w-100')
        for subtag in prof_tag_list:
            prof_name = subtag.h6.text
            contact_raw_data = subtag.div.p.text if (subtag.div != None and subtag.div.p != None) else ['', '']
            contact_raw_data = contact_raw_data.strip().split()
            contact_no, email = get_contact_info(contact_raw_data)

            data = {
                'Contact No.': contact_no,
                'Email ID': email, 
                'Research Area': area
            }
            
            final_data.append((prof_name, data))
            
        
    return dict(final_data)

def save_data_for_A_DEPT(url):
    data = get_area_and_profData(url)
    
    df = pd.DataFrame(data.values(), index = data.keys())
    df['Professor Name'] = df.index
    df['Professor Name'] = df['Professor Name'].apply(lambda x: x.strip().split('Prof.')[1])
    df.index = df['Professor Name']
    df.drop('Professor Name', axis = 1, inplace = True)
    
    file_name = (url.split('/')[-1]).upper() + '.csv'
    if os.path.exists(file_name):
        print(f'File "{file_name}" already exists. Skipping....')
        return
    else:
        df.to_csv(file_name)
        print(f'Data has been writtern into {file_name}')

In [ ]:
url = 'https://www.nitw.ac.in/bio'
save_data_for_A_DEPT(url)

## Final Function to Save Data of 'ALL Dept'

In [ ]:
def get_page_source_dept_NAME(url):
    driver = webdriver.Chrome()
    driver.get(url)
    WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'department-title')))
    page_source = driver.page_source
    return page_source

def get_dept_link(html_data):
    dept_links = []
    soup = BeautifulSoup(html_data)
    dept_link_tag_list = soup.find_all('a', class_ = 'deptviewmore')
    for tag in dept_link_tag_list:
        dept_links.append(tag['href'])
    return dept_links

def get_page_source_for_DATA2(url):
    driver = webdriver.Chrome()
    driver.get(url)
    WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'col-md-6')))

    page_source = driver.page_source
    return page_source


def get_area_and_profData(url):
    page_source = get_page_source_for_DATA2(url)
    soup = BeautifulSoup(page_source)
    
    div_tag = soup.find_all('div', class_ = 'row academic_group')
    div_tag = div_tag[1:]
    
    final_data = []
    for tag in div_tag:
        area = tag.find_all('div', class_ = 'col-md-12')[0].header.h4.text
        
        prof_tag_list = tag.find_all('div', class_ = 'ml-3 w-100')
        for subtag in prof_tag_list:
            prof_name = subtag.h6.text
            contact_raw_data = subtag.div.p.text if (subtag.div != None and subtag.div.p != None) else ['', '']
            contact_raw_data = contact_raw_data.strip().split()
            contact_no, email = contact_raw_data[0], contact_raw_data[1]

            data = {
                'Contact No.': contact_no,
                'Email ID': email, 
                'Research Area': area
            }
            
            final_data.append((prof_name, data))
            
        
    return dict(final_data)

def save_data_for_A_DEPT(url):
    data = get_area_and_profData(url)
    
    df = pd.DataFrame(data.values(), index = data.keys())

    folder_name = 'Research Faculty Data'
    if not os.path.exists(folder_name):
        os.mkdir(folder_name)
        
    file_name = (url.split('/')[-1]).upper() + '.csv'
    file_path = os.path.join(folder_name, file_name)
    
    if os.path.exists(file_path):
        print(f'File "{file_name}" already exists. Skipping....')
        return
    else:
        df.to_csv(file_path)
        print(f'Data has been writtern into "{file_name}"')
        
def save_all_research_prof_data_of_each_dept(main_url):
    page_source = get_page_source_dept_NAME(main_url)
    all_dept_links = get_dept_link(page_source)
    
    for dept_link in all_dept_links:
        save_data_for_A_DEPT(dept_link)
        
    print('----------------------------------------------')
    print('SUCCESSFULLY DONE')

In [ ]:
save_all_research_prof_data_of_each_dept('https://www.nitw.ac.in/departments')